<a href="https://colab.research.google.com/github/GDevigili/TCC-DataWarehouse/blob/load-ibge-localidades/src/localidades.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import json
import sys

sys.path.append('..')

# Data Manipulation
import pandas as pd

from utils import *

In [2]:
# # Run this cells if running in google colab
# from google.colab import data_table
# data_table.enable_dataframe_formatter()

# Data Loading

In [3]:
url = "https://servicodados.ibge.gov.br/api/v1/localidades/distritos"
request = get_request().get(url)
info = request.json()

Info é uma lista de dicionários que vem de `request.json()`

### Sample data

```py
{
  'id': 520005005,
  'nome': 'Abadia de Goiás',
  'municipio': {
    'id': 5200050,
    'nome': 'Abadia de Goiás',
    'microrregiao': {
      'id': 52010,
      'nome': 'Goiânia',
      'mesorregiao': {
        'id': 5203,
        'nome': 'Centro Goiano',
        'UF': {
          'id': 52,
          'sigla': 'GO',
          'nome': 'Goiás',
          'regiao': {
            'id': 5,
            'sigla': 'CO',
            'nome': 'Centro-Oeste'
          }
        }
      }
    },
    'regiao-imediata': {
      'id': 520001,
      'nome': 'Goiânia',
      'regiao-intermediaria': {
        'id': 5201,
        'nome': 'Goiânia',
        'UF': {
          'id': 52,
          'sigla': 'GO',
          'nome': 'Goiás',
          'regiao': {
            'id': 5,
            'sigla': 'CO',
            'nome': 'Centro-Oeste'
          }
        }
      }
    }
  }
}
```

In [4]:
m_ = info[0]
info[0]

{'id': 520005005,
 'nome': 'Abadia de Goiás',
 'municipio': {'id': 5200050,
  'nome': 'Abadia de Goiás',
  'microrregiao': {'id': 52010,
   'nome': 'Goiânia',
   'mesorregiao': {'id': 5203,
    'nome': 'Centro Goiano',
    'UF': {'id': 52,
     'sigla': 'GO',
     'nome': 'Goiás',
     'regiao': {'id': 5, 'sigla': 'CO', 'nome': 'Centro-Oeste'}}}},
  'regiao-imediata': {'id': 520001,
   'nome': 'Goiânia',
   'regiao-intermediaria': {'id': 5201,
    'nome': 'Goiânia',
    'UF': {'id': 52,
     'sigla': 'GO',
     'nome': 'Goiás',
     'regiao': {'id': 5, 'sigla': 'CO', 'nome': 'Centro-Oeste'}}}}}}

Existem localidades repetidas, necessário limpeza disso

In [5]:
for i in range(5):
  print(info[i]['municipio'])

{'id': 5200050, 'nome': 'Abadia de Goiás', 'microrregiao': {'id': 52010, 'nome': 'Goiânia', 'mesorregiao': {'id': 5203, 'nome': 'Centro Goiano', 'UF': {'id': 52, 'sigla': 'GO', 'nome': 'Goiás', 'regiao': {'id': 5, 'sigla': 'CO', 'nome': 'Centro-Oeste'}}}}, 'regiao-imediata': {'id': 520001, 'nome': 'Goiânia', 'regiao-intermediaria': {'id': 5201, 'nome': 'Goiânia', 'UF': {'id': 52, 'sigla': 'GO', 'nome': 'Goiás', 'regiao': {'id': 5, 'sigla': 'CO', 'nome': 'Centro-Oeste'}}}}}
{'id': 3100104, 'nome': 'Abadia dos Dourados', 'microrregiao': {'id': 31019, 'nome': 'Patrocínio', 'mesorregiao': {'id': 3105, 'nome': 'Triângulo Mineiro/Alto Paranaíba', 'UF': {'id': 31, 'sigla': 'MG', 'nome': 'Minas Gerais', 'regiao': {'id': 3, 'sigla': 'SE', 'nome': 'Sudeste'}}}}, 'regiao-imediata': {'id': 310061, 'nome': 'Monte Carmelo', 'regiao-intermediaria': {'id': 3111, 'nome': 'Uberlândia', 'UF': {'id': 31, 'sigla': 'MG', 'nome': 'Minas Gerais', 'regiao': {'id': 3, 'sigla': 'SE', 'nome': 'Sudeste'}}}}}
{'id'

# Preparing the Dataset

## Data Unpacking

In [6]:
# def unpack_json(json_dict: dict, row: dict = {}, parent_col = '') -> dict:
#   """
#     Make a documentation (how I wish colab had copilot :/ )
#   """
#   # Iterate over each key/value pair on the data
#   for key in json_dict:
#     # If the value is another dict, i.e. it has more data
#     if type(json_dict[key]) == dict:
#       # recursivelly call unpack_json() to extract the data
#       # passing the a json_dict, the current row (pointer), and the name of the parent col
#       unpack_json(json_dict[key], row = row, parent_col = key)
#     else:
#       # If it's a value (e.g. str, int, etc.), register it on the row
#       # formatting the row's column name
#       if parent_col == '':
#         col_name = str(parent_col) + str(key)
#       else:
#         col_name = str(parent_col) + '_' + str(key)
#       # register the value in the row
#       row[col_name] = json_dict[key]
#   return row.copy()

In [7]:
rows = []
for i in range(len(info)):
  data = unpack_json(info[i])
  rows.append(data)

## Making a Data Frame

In [8]:
df = pd.DataFrame(rows)

In [9]:
df

,id,nome,municipio-id,municipio-nome,microrregiao-id,microrregiao-nome,mesorregiao-id,mesorregiao-nome,UF-id,UF-sigla,UF-nome,regiao-id,regiao-sigla,regiao-nome,regiao-imediata-id,regiao-imediata-nome,regiao-intermediaria-id,regiao-intermediaria-nome
0,520005005,Abadia de Goiás,5200050,Abadia de Goiás,52010,Goiânia,5203,Centro Goiano,52,GO,Goiás,5,CO,Centro-Oeste,520001,Goiânia,5201,Goiânia
1,310010405,Abadia dos Dourados,3100104,Abadia dos Dourados,31019,Patrocínio,3105,Triângulo Mineiro/Alto Paranaíba,31,MG,Minas Gerais,3,SE,Sudeste,310061,Monte Carmelo,3111,Uberlândia
2,520010005,Abadiânia,5200100,Abadiânia,52012,Entorno de Brasília,5204,Leste Goiano,52,GO,Goiás,5,CO,Centro-Oeste,520002,Anápolis,5201,Goiânia
3,520010010,Posse d'Abadia,5200100,Abadiânia,52012,Entorno de Brasília,5204,Leste Goiano,52,GO,Goiás,5,CO,Centro-Oeste,520002,Anápolis,5201,Goiânia
4,310020305,Abaeté,3100203,Abaeté,31024,Três Marias,3106,Central Mineira,31,MG,Minas Gerais,3,SE,Sudeste,310070,Abaeté,3113,Divinópolis
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10665,293360415,Iguira,2933604,Xique-Xique,29006,Barra,2902,Vale São-Franciscano da Bahia,29,BA,Bahia,2,NE,Nordeste,290021,Xique-Xique - Barra,2907,Irecê
10666,251740705,Zabelê,2517407,Zabelê,25010,Cariri Ocidental,2502,Borborema,25,PB,Paraíba,2,NE,Nordeste,250007,Monteiro,2502,Campina Grande
10667,355715405,Zacarias,3557154,Zacarias,35004,São José do Rio Preto,3501,São José do Rio Preto,35,SP,São Paulo,3,SE,Sudeste,350023,Birigui - Penápolis,3506,Araçatuba
10668,211400705,Zé Doca,2114007,Zé Doca,21008,Pindaré,2102,Oeste Maranhense,21,MA,Maranhão,2,NE,Nordeste,210009,Santa Inês,2102,Santa Inês - Bacabal


# Data Exploration

Showing a sample of the data:

In [10]:
df

,id,nome,municipio-id,municipio-nome,microrregiao-id,microrregiao-nome,mesorregiao-id,mesorregiao-nome,UF-id,UF-sigla,UF-nome,regiao-id,regiao-sigla,regiao-nome,regiao-imediata-id,regiao-imediata-nome,regiao-intermediaria-id,regiao-intermediaria-nome
0,520005005,Abadia de Goiás,5200050,Abadia de Goiás,52010,Goiânia,5203,Centro Goiano,52,GO,Goiás,5,CO,Centro-Oeste,520001,Goiânia,5201,Goiânia
1,310010405,Abadia dos Dourados,3100104,Abadia dos Dourados,31019,Patrocínio,3105,Triângulo Mineiro/Alto Paranaíba,31,MG,Minas Gerais,3,SE,Sudeste,310061,Monte Carmelo,3111,Uberlândia
2,520010005,Abadiânia,5200100,Abadiânia,52012,Entorno de Brasília,5204,Leste Goiano,52,GO,Goiás,5,CO,Centro-Oeste,520002,Anápolis,5201,Goiânia
3,520010010,Posse d'Abadia,5200100,Abadiânia,52012,Entorno de Brasília,5204,Leste Goiano,52,GO,Goiás,5,CO,Centro-Oeste,520002,Anápolis,5201,Goiânia
4,310020305,Abaeté,3100203,Abaeté,31024,Três Marias,3106,Central Mineira,31,MG,Minas Gerais,3,SE,Sudeste,310070,Abaeté,3113,Divinópolis
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10665,293360415,Iguira,2933604,Xique-Xique,29006,Barra,2902,Vale São-Franciscano da Bahia,29,BA,Bahia,2,NE,Nordeste,290021,Xique-Xique - Barra,2907,Irecê
10666,251740705,Zabelê,2517407,Zabelê,25010,Cariri Ocidental,2502,Borborema,25,PB,Paraíba,2,NE,Nordeste,250007,Monteiro,2502,Campina Grande
10667,355715405,Zacarias,3557154,Zacarias,35004,São José do Rio Preto,3501,São José do Rio Preto,35,SP,São Paulo,3,SE,Sudeste,350023,Birigui - Penápolis,3506,Araçatuba
10668,211400705,Zé Doca,2114007,Zé Doca,21008,Pindaré,2102,Oeste Maranhense,21,MA,Maranhão,2,NE,Nordeste,210009,Santa Inês,2102,Santa Inês - Bacabal


In [11]:
df.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10670 entries, 0 to 10669
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   id                         10670 non-null  int64 
 1   nome                       10670 non-null  object
 2   municipio-id               10670 non-null  int64 
 3   municipio-nome             10670 non-null  object
 4   microrregiao-id            10670 non-null  int64 
 5   microrregiao-nome          10670 non-null  object
 6   mesorregiao-id             10670 non-null  int64 
 7   mesorregiao-nome           10670 non-null  object
 8   UF-id                      10670 non-null  int64 
 9   UF-sigla                   10670 non-null  object
 10  UF-nome                    10670 non-null  object
 11  regiao-id                  10670 non-null  int64 
 12  regiao-sigla               10670 non-null  object
 13  regiao-nome                10670 non-null  object
 14  regiao

#### Unique counts

In [12]:
cols_info = []
for column in df.columns:
  cols_info.append({
      'col_name': column,
      'unique_count': len(df[column].unique())
  })
  
cols_info

[{'col_name': 'id', 'unique_count': 10670},
 {'col_name': 'nome', 'unique_count': 9193},
 {'col_name': 'municipio-id', 'unique_count': 5570},
 {'col_name': 'municipio-nome', 'unique_count': 5297},
 {'col_name': 'microrregiao-id', 'unique_count': 558},
 {'col_name': 'microrregiao-nome', 'unique_count': 554},
 {'col_name': 'mesorregiao-id', 'unique_count': 137},
 {'col_name': 'mesorregiao-nome', 'unique_count': 137},
 {'col_name': 'UF-id', 'unique_count': 27},
 {'col_name': 'UF-sigla', 'unique_count': 27},
 {'col_name': 'UF-nome', 'unique_count': 27},
 {'col_name': 'regiao-id', 'unique_count': 5},
 {'col_name': 'regiao-sigla', 'unique_count': 5},
 {'col_name': 'regiao-nome', 'unique_count': 5},
 {'col_name': 'regiao-imediata-id', 'unique_count': 510},
 {'col_name': 'regiao-imediata-nome', 'unique_count': 508},
 {'col_name': 'regiao-intermediaria-id', 'unique_count': 133},
 {'col_name': 'regiao-intermediaria-nome', 'unique_count': 133}]


**OBSERVAÇÕES PARA FUTURA LIMPEZA DE DADOS:**


Existem municípios com nome repetido mas ID distintos, possibilidades:
+ Município localizado em duas micro/meso regiões/UFs distintas
+ Municípios com mesmo nome

(O mesmo se propaga para demais dados.)

Existem 5568 municípios + 2 distritos (federal e Fernando de Noronha) que compõem os 5570 id's de município

# Other Datasets

## Regiões Integradas de Desenvolvimento

In [13]:
url = 'https://servicodados.ibge.gov.br/api/v1/localidades/regioes-integradas-de-desenvolvimento'

## Subdistritos

In [14]:
url = 'https://servicodados.ibge.gov.br/api/v1/localidades/subdistritos'

## Regiões Metropolitanas

In [15]:
url = 'https://servicodados.ibge.gov.br/api/v1/localidades/regioes-metropolitanas/'

In [16]:
unpack_json(info[0])

{'id': 520005005,
 'nome': 'Abadia de Goiás',
 'municipio-id': 5200050,
 'municipio-nome': 'Abadia de Goiás',
 'microrregiao-id': 52010,
 'microrregiao-nome': 'Goiânia',
 'mesorregiao-id': 5203,
 'mesorregiao-nome': 'Centro Goiano',
 'UF-id': 52,
 'UF-sigla': 'GO',
 'UF-nome': 'Goiás',
 'regiao-id': 5,
 'regiao-sigla': 'CO',
 'regiao-nome': 'Centro-Oeste',
 'regiao-imediata-id': 520001,
 'regiao-imediata-nome': 'Goiânia',
 'regiao-intermediaria-id': 5201,
 'regiao-intermediaria-nome': 'Goiânia'}